# Flight Delay Predictor based on Earlier Flight
### This Python script loads flight data, cleans it, and performs exploratory data analysis (EDA) to understand the correlation between various factors and flight delays. It preprocesses the data, including one-hot encoding categorical variables and standardizing numerical variables. Several machine learning models are trained and evaluated to predict flight delays accurately, using earlier flight data. The best-performing model is then used to make predictions for three types of scenarios—early, on-time, and late arrivals of previous flights at a specific airport. These predictions are made with the model that achieved the highest accuracy during training.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data from the CSV file
df = pd.read_csv('cleaned_adw_df_22.csv')
cols_to_remove = ['Delay National Aviation System (Minutes)_a_e', 'Delay Security (Minutes)_a_e', 'Delay Late Aircraft Arrival (Minutes)_a_e', 
 'Delay National Aviation System (Minutes)_d_e', 'Delay Security (Minutes)_d_e', 'Delay Late Aircraft Arrival (Minutes)_d_e', 
 'Delay National Aviation System (Minutes)_a_l', 'Delay Security (Minutes)_a_l', 'Delay Late Aircraft Arrival (Minutes)_a_l', 
 'Delay National Aviation System (Minutes)_d_l', 'Delay Security (Minutes)_d_l', 'Delay Late Aircraft Arrival (Minutes)_d_l','Date (MM/DD/YYYY)_d_e']
df =df.drop(columns=cols_to_remove)
# filtered data based on year
years_to_filter = {2019,2020,2021,2022,2023}
df = df[df['Year_e'].isin(years_to_filter)]

airport = 'JFK'

df = df[df['Origin Airport_a']==airport]

df.head(3)

Origin Airport_a  Scheduled Elapsed Time (Minutes)_a_e  \
0              JFK                                  87.0   
1              JFK                                  87.0   
2              JFK                                  89.0   

   Arrival Delay (Minutes)_a_e  temp_a_e wx_phrase_a_e  dewPt_a_e  \
0                        -23.0      14.0        Cloudy        2.0   
1                        -21.0      16.0         Snowy       13.0   
2                         25.0       3.0         Snowy       -1.0   

   heat_index_a_e  rh_a_e  pressure_a_e  vis_a_e  ...  snow_hrly_d_l  \
0            14.0    59.0         30.09     10.0  ...            0.0   
1            16.0    88.0         29.61      3.0  ...            0.0   
2             3.0    84.0         29.39      1.0  ...            0.0   

   uv_desc_d_l  feels_like_d_l  uv_index_d_l clds_d_l  Main_Carrier_l  \
0          Low            24.0           0.0      BKN              B6   
1          Low            30.0           0.0      OVC              B6   
2          Low            10.0           0.0      BKN              B6   

   Week_Category_l A_Time_of_Day_l  Year_l time_diff  
0          Weekend         Evening    2019      0.88  
1          Weekend         Evening    2019      0.88  
2          Weekend         Evening    2019      0.88  

[3 rows x 74 columns]

In [2]:
# Convert categorical variables to numerical using one-hot encoding
categorical_cols = ['Origin Airport_a', 'wx_phrase_a_e', 'uv_desc_a_e', 'clds_a_e', 'wx_phrase_d_e', 'uv_desc_d_e', 
                    'clds_d_e', 'Main_Carrier_e', 'Week_Category_e', 'A_Time_of_Day_e', 'wx_phrase_a_l', 'uv_desc_a_l',
                    'clds_a_l', 'wx_phrase_d_l', 'uv_desc_d_l', 'clds_d_l', 'Main_Carrier_l', 'Week_Category_l', 'A_Time_of_Day_l']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Select only numeric columns
numeric_cols = df.select_dtypes(include=['number'])

# Calculate correlation with "Arrival Delay (Minutes)_a" and sort by absolute value
corr_with_delay = df.corr()['Arrival Delay (Minutes)_a_l'].abs().sort_values(ascending=False)

# print(corr_with_delay)

# Select top 12 features with low intercorrelation
selected_features = []
for feature in corr_with_delay.index:
    if len(selected_features) >= 30:
        break
    if corr_with_delay[feature] >= -0.1:
        selected_features.append(feature)
        
selected_features.remove('Arrival Delay (Minutes)_a_l')
print(selected_features)

# Print remaining numerical columns
remaining_numeric_cols = [col for col in numeric_cols.columns if col not in selected_features]
remaining_numeric_cols.remove('Arrival Delay (Minutes)_a_l')
print("Remaining Numerical Columns:")
print(remaining_numeric_cols)

# Remove the specified columns

columns_to_remove = remaining_numeric_cols
# .append('Year')

df = df.drop(columns=remaining_numeric_cols)

['A_Time_of_Day_l_Night', 'Arrival Delay (Minutes)_a_e', 'A_Time_of_Day_e_Night', 'rh_a_e', 'vis_a_l', 'uv_index_d_e', 'rh_a_l', 'uv_index_a_e', 'Scheduled Elapsed Time (Minutes)_a_e', 'uv_index_d_l', 'rh_d_l', 'vis_a_e', 'Year_l', 'Year_e', 'wx_phrase_d_e_Windy', 'uv_desc_a_l_Low', 'wx_phrase_d_l_Snowy', 'uv_index_a_l', 'dewPt_a_e', 'rh_d_e', 'wspd_d_e', 'wx_phrase_d_l_Windy', 'uv_desc_a_l_Moderate', 'dewPt_d_l', 'A_Time_of_Day_l_Evening Peak', 'dewPt_d_e', 'pressure_d_e', 'pressure_d_l', 'wx_phrase_a_e_Snowy']
Remaining Numerical Columns:
['temp_a_e', 'heat_index_a_e', 'pressure_a_e', 'wspd_a_e', 'precip_total_a_e', 'precip_hrly_a_e', 'snow_hrly_a_e', 'feels_like_a_e', 'temp_d_e', 'heat_index_d_e', 'vis_d_e', 'precip_total_d_e', 'precip_hrly_d_e', 'snow_hrly_d_e', 'feels_like_d_e', 'Scheduled Elapsed Time (Minutes)_a_l', 'temp_a_l', 'dewPt_a_l', 'heat_index_a_l', 'pressure_a_l', 'wspd_a_l', 'precip_total_a_l', 'precip_hrly_a_l', 'snow_hrly_a_l', 'feels_like_a_l', 'temp_d_l', 'heat_in

In [3]:
# Define the target variable based on arrival delay
df['Class of Arrival Time'] = pd.cut(df['Arrival Delay (Minutes)_a_l'],
                                       bins=[float('-inf'), -5, 5, float('inf')],
                                       labels=['Early', 'On-time', 'Late'])

# Drop unnecessary columns and the target variable
X = df.drop(['Arrival Delay (Minutes)_a_l', 'Class of Arrival Time'], axis=1)
y = df['Class of Arrival Time']
# y = df['Arrival Delay (Minutes)_a']


# Convert categorical variables to numerical using one-hot encoding
# categorical_cols = ['Origin Airport_a', 'wx_phrase_a', 'uv_desc_a', 'clds_a',
#                     'wx_phrase_d', 'uv_desc_d', 'clds_d', 'Main_Carrier', 'Week_Category', 'A_Time_of_Day']
# X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
X_encoded = X

# Standardize numerical columns
numerical_cols = selected_features
scaler = StandardScaler()
X_encoded[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y,stratify = y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
# X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

# Identify the model with the highest accuracy
best_model_name = None
best_accuracy = 0


# Initialize and train different models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42,n_estimators = 250,max_depth = 50, learning_rate= 1 ),
    'ada' : AdaBoostClassifier(random_state=42),
# 'xg':  XGBClassifier(random_state=42),
'knn' : KNeighborsClassifier(n_neighbors=15),
'dtree':  DecisionTreeClassifier(max_depth=7,random_state=42),
'rand_class':  RandomForestClassifier(max_depth=15,random_state=42),
'GNB':  GaussianNB()
}

for model_name, model in models.items():

    model.fit(X_train, y_train)
    if model_name =='Logistic Regression':
        print(model.coef_,model.intercept_)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name} Accuracy: {accuracy:.2f}')
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name
    # mae = mean_absolute_error(y_test, y_pred)
    # mae_ratio = mae / np.mean(y_test)
    # print("MAE:", mae)
    # print("MAE Ratio:", mae_ratio)
    # print(model.score(X_test,y_test))
    # print(classification_report(y_test, y_pred))
    # print('-' * 50)


# Select the best model
best_model = models[best_model_name]

print("model with most accuracy is " + str(best_model))

[[-0.53519898 -0.43807629  0.11754523 -0.24632484 -0.09944515  0.33091552
  -0.0720768   0.01210269 -0.29058939  0.0229972  -0.39878598 -0.23355254
   0.13125732  0.13550677 -0.21049919  0.1320161  -0.01982817  0.28151622
  -0.01319836 -0.23355254  0.14151038 -0.13559005 -0.12162876 -0.15857092
   0.23554474  0.00853659  0.17111524 -0.22451235 -0.05699533 -0.01083921
  -0.07042987  0.09503806  0.62734637 -0.11204562 -0.3260023  -0.56607439
  -0.19167863 -0.41809979  0.11476695 -0.33160085 -0.4757102  -0.06133194
  -0.08316548 -0.11151203 -0.03861199 -0.016251    0.19283762  0.33646962
  -0.01853485 -0.53768185 -0.02716975 -0.16947419 -0.41785435 -0.05657542
   0.40585206 -0.3146629   0.63431985 -0.56795399 -0.12102847 -0.11641524
   0.16570801  0.01598721 -0.03571545 -0.34391231 -0.22106908 -0.21964456
  -0.26013234 -0.19715995 -0.1795826   0.71065688  0.00205099  0.02593084
  -0.33960077 -0.2208968   0.40732134 -0.03861199 -0.016251    0.1808581
  -0.55621669 -0.23583941]
 [ 0.2286010

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Gradient Boosting Accuracy: 0.54
ada Accuracy: 0.57
knn Accuracy: 0.62
dtree Accuracy: 0.56
rand_class Accuracy: 0.62
GNB Accuracy: 0.60
model with most accuracy is RandomForestClassifier(random_state=42)


In [5]:

# Load the data from the CSV file
df = pd.read_csv('CIS_662_FINAL_Predictions.csv')

# Filter rows with ORIGIN as JFK
airport_data_index = df[df['Origin Airport_a'] == 'JFK'].index

# Change ARRIVAL STATUS_Prev_flight to 'early' and make predictions
df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight'] = 'early'

# Perform one-hot encoding on the JFK data using previous encoder
airport_data_encoded_early = pd.get_dummies(df.loc[airport_data_index], columns=categorical_cols, drop_first=True)

# Standardize numerical columns using previous scaler
airport_data_encoded_early[numerical_cols] = scaler.transform(airport_data_encoded_early[numerical_cols])

# Make predictions using the selected model
predictions_early = best_model.predict(airport_data_encoded_early[X_train.columns])

df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight_early'] = predictions_early

print(airport + " airport prediction for ARRIVAL STATUS_Prev_flight_early are created")


# Change ARRIVAL STATUS_Prev_flight to 'ontime' and make predictions
df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight'] = 'ontime'

# Perform one-hot encoding on the JFK data using previous encoder
airport_data_encoded_ontime = pd.get_dummies(df.loc[airport_data_index], columns=categorical_cols, drop_first=True)

# Standardize numerical columns using previous scaler
airport_data_encoded_ontime[numerical_cols] = scaler.transform(airport_data_encoded_ontime[numerical_cols])

# Make predictions using the selected model
predictions_ontime = best_model.predict(airport_data_encoded_ontime[X_train.columns])


df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight_ontime'] = predictions_ontime

print(airport + " airport prediction for ARRIVAL STATUS_Prev_flight_ontime are created")



# Change ARRIVAL STATUS_Prev_flight to 'late' and make predictions
df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight'] = 'late'

# Perform one-hot encoding on the JFK data using previous encoder
airport_data_encoded_late = pd.get_dummies(df.loc[airport_data_index], columns=categorical_cols, drop_first=True)

# Standardize numerical columns using previous scaler
airport_data_encoded_late[numerical_cols] = scaler.transform(airport_data_encoded_late[numerical_cols])

# Make predictions using the selected model
predictions_late = best_model.predict(airport_data_encoded_late[X_train.columns])

df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight_late'] = predictions_late

print(airport + " airport prediction for ARRIVAL STATUS_Prev_flight_late are created")


# Save the updated DataFrame back to the same file
df.to_csv('CIS_662_FINAL_Predictions.csv', index=False)


JFK airport prediction for ARRIVAL STATUS_Prev_flight_early are created
JFK airport prediction for ARRIVAL STATUS_Prev_flight_ontime are created
JFK airport prediction for ARRIVAL STATUS_Prev_flight_late are created
